In [1]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [2]:
data  = pd.read_excel('./data/Sample - Superstore.xls')

In [3]:
data.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country/Region,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2020-152156,2020-11-08,2020-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2020-152156,2020-11-08,2020-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2020-138688,2020-06-12,2020-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2019-108966,2019-10-11,2019-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2019-108966,2019-10-11,2019-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Row ID          9994 non-null   int64         
 1   Order ID        9994 non-null   object        
 2   Order Date      9994 non-null   datetime64[ns]
 3   Ship Date       9994 non-null   datetime64[ns]
 4   Ship Mode       9994 non-null   object        
 5   Customer ID     9994 non-null   object        
 6   Customer Name   9994 non-null   object        
 7   Segment         9994 non-null   object        
 8   Country/Region  9994 non-null   object        
 9   City            9994 non-null   object        
 10  State           9994 non-null   object        
 11  Postal Code     9983 non-null   float64       
 12  Region          9994 non-null   object        
 13  Product ID      9994 non-null   object        
 14  Category        9994 non-null   object        
 15  Sub-

In [5]:
data.describe()

,Row ID,Postal Code,Sales,Quantity,Discount,Profit
count,9994.000000,9983.000000,9994.000000,9994.000000,9994.000000,9994.000000
mean,4997.500000,55245.233297,229.858001,3.789574,0.156203,28.656896
std,2885.163629,32038.715955,623.245101,2.225110,0.206452,234.260108
min,1.000000,1040.000000,0.444000,1.000000,0.000000,-6599.978000
25%,2499.250000,23223.000000,17.280000,2.000000,0.000000,1.728750
50%,4997.500000,57103.000000,54.490000,3.000000,0.200000,8.666500
75%,7495.750000,90008.000000,209.940000,5.000000,0.200000,29.364000
max,9994.000000,99301.000000,22638.480000,14.000000,0.800000,8399.976000


In [6]:
#датасет в котором содержаться данные после начала 2020 года в центральном регионе
data_2020_and_2021_central_region = data[(data['Order Date'] > '2020-01-01') & (data['Region'] == 'Central')]

In [7]:
data_2020_and_2021_central_region['Segment'].value_counts()

Consumer       697
Corporate      415
Home Office    269
Name: Segment, dtype: int64

In [8]:
data_2020_and_2021_central_region['Sub-Category'].value_counts()

Binders        212
Paper          201
Furnishings    126
Phones         120
Storage        120
Art            115
Accessories    111
Chairs          88
Appliances      74
Labels          45
Tables          43
Fasteners       33
Envelopes       30
Bookcases       28
Supplies        22
Machines         9
Copiers          4
Name: Sub-Category, dtype: int64

In [9]:
# Продажи сгрупированные по сегментам в 2020 году
data_2020_and_2021_central_region['sales_2020_by_segment'] = data_2020_and_2021_central_region[
    (data_2020_and_2021_central_region['Order Date'] < '2021-01-01')].groupby(by = 'Segment')['Sales'].transform('sum')

# Продажи сгрупированные по сегментам в 2021 году
data_2020_and_2021_central_region['sales_2021_by_segment'] = data_2020_and_2021_central_region[
    (data_2020_and_2021_central_region['Order Date'] >= '2021-01-01')].groupby(by = 'Segment')['Sales'].transform('sum')

In [10]:
#Сводная таблица для определения быстрорастущего сегмента
most_increased_seg = data_2020_and_2021_central_region.pivot_table( ['sales_2020_by_segment', 'sales_2021_by_segment'],'Segment')

most_increased_seg['increase'] = ((most_increased_seg.sales_2021_by_segment - most_increased_seg.sales_2020_by_segment)) #Увеличение прибыли в 2021 году по отношению к 2020
most_increased_seg['percent'] = ((most_increased_seg.increase / most_increased_seg.sales_2020_by_segment * 100)) # Процент увеличения продаж в 2021 году по отношению 2020

most_increased_seg.sort_values(by='increase', ascending = False)

,sales_2020_by_segment,sales_2021_by_segment,increase,percent
Segment,,,,
Home Office,20008.2290,30452.4194,10444.1904,52.199475
Consumer,66707.0332,68345.6754,1638.6422,2.456476
Corporate,60714.1138,48300.0334,-12414.0804,-20.446779


In [11]:
#Продажи сгрупированные подкатегориях за 2020 и 2021 год
data_2020_and_2021_central_region['Sales_2020_by_sub_cat'] = data_2020_and_2021_central_region[(data_2020_and_2021_central_region['Order Date'] < '2021-01-01')
                                                                                               & (data['Segment'] == 'Home Office')].groupby(by = 'Sub-Category')['Sales'].transform('sum')
data_2020_and_2021_central_region['Sales_2021_by_sub_cat'] = data_2020_and_2021_central_region[(data_2020_and_2021_central_region['Order Date'] >= '2021-01-01')
                                                                                               & (data['Segment'] == 'Home Office')].groupby(by = 'Sub-Category')['Sales'].transform('sum')
#Прибыль сгрупированная подкатегориях за 2020 и 2021 год
data_2020_and_2021_central_region['Profit_2020_by_sub_cat'] = data_2020_and_2021_central_region[(data_2020_and_2021_central_region['Order Date'] < '2021-01-01')
                                                                                                & (data['Segment'] == 'Home Office')].groupby(by = 'Sub-Category')['Profit'].transform('sum')
data_2020_and_2021_central_region['Profit_2021_by_sub_cat'] = data_2020_and_2021_central_region[(data_2020_and_2021_central_region['Order Date'] >= '2021-01-01')
                                                                                                & (data['Segment'] == 'Home Office')].groupby(by = 'Sub-Category')['Profit'].transform('sum')
#Колличество покупок сгрупированные  в подкатегориях за 2020 и 2021 год
data_2020_and_2021_central_region['Quantity_2020_by_sub_cat'] = data_2020_and_2021_central_region[(data_2020_and_2021_central_region['Order Date'] < '2021-01-01')
                                                                                                  & (data['Segment'] == 'Home Office')].groupby(by = 'Sub-Category')['Quantity'].transform('sum')
data_2020_and_2021_central_region['Quantity_2021_by_sub_cat'] = data_2020_and_2021_central_region[(data_2020_and_2021_central_region['Order Date'] >= '2021-01-01')
                                                                                                  & (data['Segment'] == 'Home Office')].groupby(by = 'Sub-Category')['Quantity'].transform('sum')

In [12]:
data_2020_and_2021_central_region['Quantity']

21      7
22      7
34      3
35      7
36      5
       ..
9950    1
9951    2
9972    2
9982    7
9983    2
Name: Quantity, Length: 1381, dtype: int64

In [13]:
#Сводная таблица для подкатегорий товаров которые увеличиваются по продажам но уменьшаются по среднему чеку 
sub_cat = data_2020_and_2021_central_region.pivot_table( ['Sales_2020_by_sub_cat', 'Sales_2021_by_sub_cat','Profit_2020_by_sub_cat',
                                                          'Profit_2021_by_sub_cat','Quantity_2020_by_sub_cat','Quantity_2021_by_sub_cat'],'Sub-Category')
sub_cat = sub_cat.fillna(0)
sub_cat['increase_profit'] = (sub_cat.Profit_2021_by_sub_cat - sub_cat.Profit_2020_by_sub_cat) # Увеличени прибыли в 2021 по сравнению с 2020 годом
sub_cat['increase_sales'] = (sub_cat.Sales_2021_by_sub_cat - sub_cat.Sales_2020_by_sub_cat) # Увеличени продаж в 2021 по сравнению с 2020 годом
sub_cat['mean_check_2020'] = (sub_cat.Sales_2020_by_sub_cat / sub_cat.Quantity_2020_by_sub_cat) # Средний чек в 2020
sub_cat['mean_check_2021'] = (sub_cat.Sales_2021_by_sub_cat / sub_cat.Quantity_2021_by_sub_cat) # Средний чек в 2021

sub_cat = sub_cat.fillna(0)
sub_cat = sub_cat.sort_values(by=['Quantity_2021_by_sub_cat'], ascending = False)
sub_cat

,Profit_2020_by_sub_cat,Profit_2021_by_sub_cat,Quantity_2020_by_sub_cat,Quantity_2021_by_sub_cat,Sales_2020_by_sub_cat,Sales_2021_by_sub_cat,increase_profit,increase_sales,mean_check_2020,mean_check_2021
Sub-Category,,,,,,,,,,
Binders,93.3024,-482.1276,54.0,105.0,406.120,2936.6120,-575.4300,2530.4920,7.520741,27.967733
Paper,754.4257,544.5043,60.0,81.0,1686.674,1391.8380,-209.9214,-294.8360,28.111233,17.183185
Phones,760.5102,1669.4762,28.0,65.0,3249.176,9081.5360,908.9660,5832.3600,116.042000,139.715938
Storage,-73.4759,62.7396,44.0,58.0,2588.078,2636.6280,136.2155,48.5500,58.819955,45.459103
Art,68.1978,95.5262,36.0,55.0,243.828,391.6600,27.3284,147.8320,6.773000,7.121091
Accessories,81.5606,1391.0628,37.0,40.0,1322.278,4593.9840,1309.5022,3271.7060,35.737243,114.849600
Chairs,-110.9396,20.2784,19.0,39.0,2743.908,3093.3140,131.2180,349.4060,144.416211,79.315744
Furnishings,-851.2000,-163.5327,54.0,35.0,1018.824,866.5100,687.6673,-152.3140,18.867111,24.757429
Appliances,938.2637,160.5926,30.0,27.0,3832.894,1069.0580,-777.6711,-2763.8360,127.763133,39.594741


In [14]:
# список подкатегорий которые увеличиваются по продажам но уменьшаются по среднему чеку 
sub_cat[(sub_cat.Quantity_2021_by_sub_cat  > sub_cat.Quantity_2020_by_sub_cat ) & (sub_cat.mean_check_2021 < sub_cat.mean_check_2020)]

,Profit_2020_by_sub_cat,Profit_2021_by_sub_cat,Quantity_2020_by_sub_cat,Quantity_2021_by_sub_cat,Sales_2020_by_sub_cat,Sales_2021_by_sub_cat,increase_profit,increase_sales,mean_check_2020,mean_check_2021
Sub-Category,,,,,,,,,,
Paper,754.4257,544.5043,60.0,81.0,1686.674,1391.838,-209.9214,-294.836,28.111233,17.183185
Storage,-73.4759,62.7396,44.0,58.0,2588.078,2636.628,136.2155,48.550,58.819955,45.459103
Chairs,-110.9396,20.2784,19.0,39.0,2743.908,3093.314,131.2180,349.406,144.416211,79.315744
Tables,319.6386,-300.4776,7.0,17.0,1390.100,1865.536,-620.1162,475.436,198.585714,109.737412
Envelopes,56.4597,60.7420,4.0,13.0,167.288,177.600,4.2823,10.312,41.822000,13.661538
Machines,43.1991,205.9557,3.0,8.0,1007.979,897.469,162.7566,-110.510,335.993000,112.183625
